Note: you may need to install the following packages to run this notebook:

1. Shapely -- see https://pypi.org/project/Shapely/ for installation instructions.
2. Descartes -- see https://docs.descarteslabs.com/installation.html for installation instructions.

In [ ]:
# Useful imports
%load_ext autoreload
%autoreload 2
%matplotlib inline
from __future__ import division
from matplotlib import pyplot as plt
import numpy as np
#import yaml
from shapely.geometry import Point, Polygon, LineString, box
from environment import Environment, plot_environment, plot_line, plot_poly, random_environment
import random as random


# Aerial Delivery and Research Support

## **The Problem** 

You lead the operations team for a group of researchers operating at remote locations throughout the Denali Wilderness. From your base camp in Denali National Park, you are responsible for delivering food, medical supplies, and equipment to the research teams located at 4 different locations throughout the wilderness--they endeavor to work at these locations throughout the summer. You are also responsible for supporting their research requirements to include maintenance of scientific equipment.

*Add in some cool pictures*

The terrain in Denali is extremely rugged and consists of a mix of taiga forests, high tundra, glaciers, and mountains. The research teams operate in locations unreachable by road and the scientists get in place by hiking in and out of their working locations. Currently supplies are delivered in the same way. This makes maintaining a continuous research presence extremely difficult as weather events, unexpected supply shortages, and other operational issues often lead to dangerously low provisions at the research sites--forcing the scientists to return intermittently to ensure the safety of the team.

Fortunately your team has just come into possession of 7 drones who are capable of delivering supplies to the teams. Your task is to:

<ol>
    <li><b> Plan routes between all sites and the base camp avoiding designated no-fly areas.</b></li>
    <li><b> Develop a supply delivery schedule based upon your planned routes, your drone delivery capabilities, and research team needs.</b></li>
    <li><b> Develop a method to diagnose scientific equipment faults and issues to determine when equipment supplies need to be delivered to a team.<b/></li>
</ol>

## **Route Planning with RRT and RRT*** 

Your drones can overfly most of terrain between you and the research teams but must take care to avoid areas designated noise sensitive due to wildlife, ecologically sensitive areas where it is deemed completely unnacceptable for a drone to crash, and campsites in the national park near your launch location. All of these areas were designated no-fly areas by the Bureau of Land Management and the National Park Service. Finally, some high altitude areas cannot be overflown due to drone performance limitations.

Your team has developed a map depicting these no-fly areas, your base camp, and the research sites. Anything outside the map boundaries is also considered a no-fly area. Based on the ecological sensitivity of Denali (and the importance that the National Park Service puts on keeping the wilderness as pristine as possible!) there are <ins> a lot </ins> of no-fly areas. Your drones can be relied upon to stay within 100 meters of any planned path (this navigation error will be exceeded with a probability of 10<sup>-5</sup> in any given flight-hour so it's acceptable use this number for path planning!).

Due to the high latitude of your operations, your team uses a grid system to specify camp locations referenced to True North. For the remote camps, you are provided with a landing zone (LZ) geometry. Each unit digit on the grid system represents one kilometer. Hence, location (-4,-2) is one kilometer *true* south of position (-3,-2).

Based on this grid system, the relevant locations are:

<ul class="dashed">
  <li>The base camp drone launch site is located at position (-2,-2)</li>
  <li>The site 1 LZ bounds are (-4.5,3.6), (-4.5,4), (-4.2,4),(-4.2,3.6).</li>
  <li>The site 2 LZ bounds are (9.7,0), (9.7,0.4), (10,0.4),(10,0).</li>
   <li>The site 3 LZ bounds are (12.7,3.6), (12.7,4), (13,4),(13,3.6).</li>
   <li>The site 4 LZ bounds are (13.6,-3.2), (13.6,-3.5), (14,-3.5),(14,-3.2).</li>
</ul>

**The code block below shows the map of the no-fly areas (in blue), the base camp (in magenta), and the LZs (in green).**

In [ ]:
# Plot the environment 
env = Environment('Denali_650.yaml')
ax = plot_environment(env)

# Base camp location
radius = 0.1
start_pose = (-2,-2)
start_point = Point(start_pose[0],start_pose[1])
start_ball = start_point.buffer(radius)

# Camp locations
LZ1 = Polygon([(-4.5,3.6), (-4.5,4), (-4.2,4),(-4.2,3.6)])
LZ2 = Polygon([(9.7,0), (9.7,0.4), (10,0.4),(10,0)])
LZ3 = Polygon([(12.7,3.6), (12.7,4), (13,4),(13,3.6)])
LZ4 = Polygon([(13.6,-3.2), (13.6,-3.5), (14,-3.5),(14,-3.2)])

LZs = (LZ1, LZ2, LZ3, LZ4)


# Plot base camp in magenta
plot_poly(ax, start_ball,'magenta')
# Plot research camps in green
for LZ in LZs:
    plot_poly(ax, LZ,'green')

<ins>**Rapidly Exploring Random Trees (RRT)**</ins>

Looking at the map, you are a bit baffled in how you can make sure you find the best routes through this maze. Fortunately, you took 16.413 at MIT and recall learning about sampling-based planning methods--in particular Rapidly Exploring Random Trees (RRT). You decide to first tackle this problem with RRT. We chose RRT over graph search primarily due to the curse of dimensionality as highlighted in the course. For a better path, we would like to discretize our map further which exponentially increases the node size in case of a graph search algoithm. Further, we can also incoporate our agent's dynamics using RRT since we simulate forward (much harder to implement in probabilistic roadmap planning).

**Academic description of RRT**
In RRT (Rapidly Exploring Random Trees), the search tree is incrementally generated using samples from search space in a random fashion. In the iterative process:
<li> A random coordinate is chosen in free space </li>
<li> Find the nearest vertex exisiting in the tree to that coordinate</li>
<li> Steer that vertex in the direction of the coordinate by a depth d</li>
<li> If the new vertex is obstacle free, add it to our tree.</li>
<li> Repeat the above steps until the vertex lands in the goal region</li>
RRT is probabilistically complete which means that it will find a solution in case the solution exists (though it may take a long time to run). A few downsides of RRT are that it usually returns a suboptimal path and also fails to check if a path does not exist (i.e. the algorithm will run forever in case the path between start and goal location does not exist).

**RRT Demo going from Base Camp --> LZ1, discuss how non-optimal**

Taking a look at the produced route, you can tell by inspection that the route is clearly not optimal. Thankfully, you remember that there is an optimal version of RRT, RRT* that you should be able to implement for this problem.

<b> Academic description of RRT* </b>
RRT* is an optimized version of RRT, and is asymptotically optimal. In an ideal case when number of nodes tend to infinity, RRT* yields the shortest path (optimal path) to the goal by keeping track of the distance each vertex has travelled relative to the parent vertex. Another variation is that after a vertex is added to the lowest cost neighbor, the tree is rewired to the new vertex in order to decrease individual cost. In the the iterative process:
<li> A random coordinate is chosen in free space </li>
<li> Find the nearest vertex exisiting in the tree to that coordinate</li>
<li> Steer that vertex in the direction of the coordinate by a depth d</li>
<li> "Still working on this" - Aadi</li>

## **Solution Development and Topics References** 

We want to use 3 tools learned in class. The RRT/RRT* probabilistic planning, constraint programming (MIP), and hidden markov models (HMM) methods for the same. The basic idea is to use the RRT/RRT* to determine the shortest distance from the base camp to each of the destinations and among all destinations. This distance will be later used as an input into the mathematical program for constraint programming. We will use an HMM to determine if scientific equipment is malfunctioning.

Once the distance matrix is populated using RRT, we will go ahead and use mathematical programming to model the entire problem as a vechile routing program to optimize the number of vehicles to be used, the type of vehicle to be used, the sequence of destinations to be allocated and the total cost of operations. (# TOM PLEASE ADD HERE)

### **Tools to be used** 

RRT - Use assignment modules with some modification <br>
Linear Programming - Use PuLP package in python (do you guys want to use Gurobi???)
HMM - code it up

## **Solution Start**

### **RRT**

**Input** : The map of the delivery region and the coordinates of the basecamp and the five destinations <br>
**Output** : The distance matrix between basecamp and destinations <br>
**Method** : Run RRT/RRT* among all points (basecamp and destinations) and populate with the shortest distance <br>

### **Mathematical Programming**

**Objective function** : <br>
The objective function is to minimize the total cost of operations. <br> 
Out intention is to have 2 components to total cost. <br>
Fixed cost : Fixed cost of using the vehicle.<br>
Variable Cost : Cost incurred by using the vehicle. This cost varies with the usage ( such as fuel costs ).<br>

**Parameters or Input to the Program** : <br>
1. Distance matrix from RRT <br> 
2. Fixed cost of using the truck USD/ trip <br>
3. Variable cost of using the truck ~ USD/ KM <br>
4. Averge speed of the vehicle ~ km/hr <br>
5. Weekly demand from each destination in terms of tons (tons) and volume (cubic meter)<br>
6. Capacity of truck in terms of tons and volume<br>
7. Eficiency of vehicle could be used ( % of weight or volume capacity could be used)<br>

**Decisions Variable** : <br>
1. X refers to the % of load of weekly demand moved from basecamp to differnet destinations ~ lies between 0 and 1 <br> 
2. Y refers to whether a truck is used or not ~ binary <br>


**Constraints** : <br>
1. No movement from destination i to detination i
2. Relationship between Y and X . If X > 0 for a trip in a day , then Y should be 1 for that trip, 0 otherwise
3. Sum across all X should be more than 100%. Weekly demand is met. 
4. Load in a truck within the capacity
5. Subtour elimination constraints ( to ensure that vehicle does not get into cycles and get stuck)
6. Balacning constraint ( at every destination what comes in goes out )


###### Below this line is only trials #

I can type **bold** and *italicized* text.

I can type equations:

$$\sum_{i=0}^{k}x_i$$

I can type in-line equations $x+y=z$.

In [4]:
import numpy as np
from pulp import *

ModuleNotFoundError: No module named 'pulp'

In [ ]:
# Define the maps containing start and goal nodes alog with obstacles

In [ ]:
# Use RRT/RRT* to calculate the optimal distance matrix coresponding to the distances between basecamp and destinations

In [ ]:
# Defining Inputs to the model ( Constants or Parameters ) 

In [6]:


n_drop_points = 10
n_drones = 5 # we need to define the drones in capacity with different
n_property = 2 # weight carrying capacity for medicines and number of vaccines in terms of nos ( cold storage )
n_trips = 5 # number of trips in a week possible 

# all sets definition
# set suppliers:= 1..n_suppliers;
# set trips:=1..n_trips;
# set truck:=1..n_trucks;
# set property := 1..n_property;
# set ARCS := {i in suppliers, j in suppliers : i == j} ; 

#param distance {suppliers,suppliers}; #need to put matrix of distances . 
distance = [[0,419,310,25,49,26],
            [419,0,592,445,483,413],
            [310,592,0,335,358,288],
            [25,445,335,0,30,51],
            [49,483,358,30,0,75],
            [26,413,288,51,75,0],
            ]
    
#param FCT {truck};
FCT = [542,577,577] # this is the fixed cost of running the drone

#param VC {truck};
VC = [32,28,28] # this is the variable cost of running the drone per km

#param S;
S = 35 #speed of the drone

#param demand {suppliers,property};
demand = [[0,0],[33.8,27.5],[10,25],[3.5,10],[7.9,11.25],[0.75,5]]
        

#param teff;
eff = 0.95 # hos much of the drone capacity we can use

#param capacity {truck,property}; # how to put drone capacity here
capacity = [[14.5,37],[7.6,37],[7.6,37]]

tripCost = [100,100,100,100,100]

max_dist = 3200  #you dont want the drone to travel more than a ptricual distance than 16 hours a week. 

profit = 1.375 #Percentage margin

# Sample Pup Problem Below for reference

nSuppliers = 5 # 0...4
nTrips = 5 # 0 ... 4
nTrucks = 5
nProp = 2

# Suppliers:
Supp = [i for i in range(nSuppliers)]
Trips = [i for i in range(nTrips)]
Trucks = [i for i in range(nTrucks)]
Property = [i for i in range(nProp)]
truck_property = [(14.5,37), (7.6,37), (7.6,37)] # (weight, volume)

In [ ]:
iter = [(i,j,d,t) for t in range(nTrucks) for d in range(nTrips) for j in range(nSuppliers) for i in range(nSuppliers)]
# Decision variables
X_ijdt = LpVariable.dicts("X",(Supp,Supp, Trips, Trucks),0,1)
Y_dt = LpVariable.dicts("Y",(Trips, Trucks),0,1, cat = 'Binary')
Z_ijdt = LpVariable.dicts("Z",(Supp,Supp, Trips, Trucks),0,1, cat = 'Binary')
R_idt = LpVariable.dicts("R",(Supp, Trips, Trucks),0,None, cat = 'Integer')

In [ ]:
# Objective funtion
B = LpProblem("Transportation_variable_cost_problem",LpMinimize)
B += lpSum([Z_ijdt[i][j][d][t]*distance[i][j]*VC[t] for (i,j,d,t) in iter]), "Sum_ofVariable_Cost"

In [ ]:
# Constraints 0,1,2
for i in Supp:
    
    for j in Supp:
            for t in Trucks:
                for d in Trips:
                    if i == j:
                        B += X_ijdt[i][j][d][t] <= 0 # Constraint 0
                    B += Y_dt[d][t] >= X_ijdt[i][j][d][t] # Constraint 1
                    B += Z_ijdt[i][j][d][t] >= X_ijdt[i][j][d][t] # Constraint 2
                     
        

In [ ]:
#constraint 3
for i in supp:
    B += lpsum([ X_ijdt[i][j][d][t] for j in Supp for d in Trips for t in Trucks ] ) >= 1 

In [ ]:
#constraint 4
for d in Trips:
    for t in Trucks:
        for p in Property:
            B += lpsum([ X_ijdt[i][j][d][t] * demand[i][p] for i in Supp, for j in Supp  ] ) <= eff * capacity[t][p]

In [ ]:
#constraint 5
for t in Trucks:
    B += lpsum([ Z_ijdt[i][j][d][t] * distance[i][j] for i in Supp, for j in Supp, for d in Trips ] ) <= max_dist
    

In [8]:
#constraint 6
for i>=2 in supp:
    for j>=2 in supp:
        B += R[j][d][t] >= R[i][d][t] - nSuppliers * ( 1- Z_ijdt[1][j][d][t])

SyntaxError: invalid syntax (<ipython-input-8-d1ef91bb6ea5>, line 2)

In [ ]:
#constraint 7
for d in Trips:
    for t in Trucks:
        B += R[1,d,t] = 1

In [ ]:
#constraint 8
for d in Trips:
    for t in Trucks:
        B += lpsum([ Z_ijdt[1][j][d][t] for j in Supp] ) == 1

In [7]:
#constraint 9
for d in Trips:
    for t in Trucks:
        for h!=1 in supp:
            B += lpsum([ Z_ijdt[i][h][d][t] for i in Supp] ) == lpsum([ Z_ijdt[h][j][d][t] for j in Supp] ) 

SyntaxError: invalid syntax (<ipython-input-7-1ae3a3bc32b1>, line 4)

# Sample Pulp problem below

In [ ]:
x1 = LpVariable("ChickenPercent", 0, None, LpInteger)
x2 = LpVariable("BeefPercent", 0)


In [ ]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("The Whiskas Problem", LpMinimize)

In [ ]:
# The objective function is added to 'prob' first
prob += 0.013 * x1 + 0.008 * x2, "Total Cost of Ingredients per can"

In [ ]:
# The five constraints are entered
prob += x1 + x2 == 100, "PercentagesSum"
prob += 0.100 * x1 + 0.200 * x2 >= 8.0, "ProteinRequirement"
prob += 0.080 * x1 + 0.100 * x2 >= 6.0, "FatRequirement"
prob += 0.001 * x1 + 0.005 * x2 <= 2.0, "FibreRequirement"
prob += 0.002 * x1 + 0.005 * x2 <= 0.4, "SaltRequirement"

In [ ]:
# The problem data is written to an .lp file
prob.writeLP("WhiskasModel.lp")

In [ ]:
# The problem is solved using PuLP's choice of Solver
prob.solve()

In [ ]:
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

In [ ]:
# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

In [ ]:
# The optimised objective function value is printed to the screen
print("Total Cost of Ingredients per can = ", value(prob.objective))

# Scientific Equipment Repair

Now that your drone shipments are up and running with an optimized schedule, your team has more capacity to help the forward deployed scientists troubleshoot their equipment.

One of the instruments they use is equipped with analog components that are prone to malfunctioning. The research teams often struggle to efficiently identify the faulty component(s) in their system when observing anomalous behavior.

A schematic of one such system is shown below.

<img src=Logic_Array.png style="width: 60%;">

According to the component manufacturer, the AND gates have a probability of failure of 2% and the exclusive OR gates have a failure probability of 2.5%.


##